In [45]:
# Importing Libraries
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from urllib.parse import urlencode


In [42]:
def crawler():
    """This function crawls nepstock website and returns some data.

    Returns:
        [type]: [description]
    """
    try:
        html = urlopen('http://www.nepalstock.com')
    except HTTPError as e:
        print(e)
    except URLError as e:
        print('The server could not be found!')
    else:
        print('Website Successfully Crawled.')
    bs = BeautifulSoup(html.read(), 'html.parser')
    
    index = bs.find('div', {'class':{'current-index'}}).text.split()[0]
    market_status = bs.find('div',{'id':'market_info'}).find('div',{'class':'top_marketinfo'}).text.strip()
    market_info = [{data.text.split('|')[0]:data.text.split('|')[1]} for data in bs.find('div',{'id':'market_info'}).find_all('span')]

    tables = bs.find('div', {'id':{'nepse-stats'}}).find_all('table', {'class':{'table table-hover table-condensed'}})

    alldata=[]
    for table in tables:
        t={}
        for row in table.find("thead").findAll('tr'):
            t["head"]=[data.text for data in row.find_all('td') if data.text !='']
            
        row_data=[]
        for row in table.find("tbody").findAll('tr'):
            col_data={} if len(t["head"])>3 else []
            for index,data in enumerate(row.find_all('td')):
                text=data.text.split()
                if len(t["head"])>3:
                    if len(text) ==1 and text[0]!='' :
                        col_data[t["head"][index]]=text[0]
                    if len(text) >1:
                        col_data[t["head"][index]]=' '.join(text)
                else:
                    if len(text) ==1 and text[0]!='' :
                        col_data.append(text[0])
                    if len(text) >1:
                        col_data.append(' '.join(text))
            if len(col_data) >1:
                row_data.append(col_data)   
            
        t["body"]=row_data 
        alldata.append(t)  
        
    # print(alldata,index)
    return {
        "NEPSE_INDEX":index,
        "market_status":market_status,
        "market_info":market_info,
        "market_data":alldata
    }   

crawler()


Website Successfully Crawled.
3


{'NEPSE_INDEX': 0,
 'market_status': 'Market Closed',
 'market_info': [{'Share Volume ': ' 8,776,337'},
  {'Turnover ': ' 2,966,899,791'}],
 'market_data': [{'head': ['Market Summary'],
   'body': [['Total Turnover Rs:', '2,966,899,791'],
    ['Total Traded Shares', '8,776,337'],
    ['Total Transactions', '40,401'],
    ['Total Scrips Traded', '218'],
    ['Total Market Capitalization Rs.', '3,623,258.42 Millions'],
    ['Floated Market Capitalization Rs.', '1,290,365.30 Millions']]},
  {'head': ['Index', 'Current', 'Points Change', '%Change'],
   'body': [{'Index': 'NEPSE',
     'Current': '2,587.45',
     'Points Change': '69.51',
     '%Change': '2.62'},
    {'Index': 'Sensitive',
     'Current': '484.93',
     'Points Change': '12.43',
     '%Change': '2.5'},
    {'Index': 'Float',
     'Current': '181.84',
     'Points Change': '5',
     '%Change': '2.68'},
    {'Index': 'Sen. Float',
     'Current': '163.98',
     'Points Change': '4.42',
     '%Change': '2.63'}]},
  {'head': ['

In [106]:
def share_price_crawler():
    """This function crawls nepstock website to get all the share prices.

    Returns:
        [type]: [description]
    """
    try:
        data = urlencode({"_limit": "300"})
        data = data.encode('ascii')
        url = "http://www.nepalstock.com/todaysprice"
        html = urlopen(url, data)
    except HTTPError as e:
        print(e)
    except URLError as e:
        print('The server could not be found!')
    else:
        print('Website Successfully Crawled.')
    bs = BeautifulSoup(html.read(), 'html.parser')
    table = bs.find('table', {'class':{'table table-condensed table-hover'}})
    rows= [row for row in table.find_all('tr') if len(row.find_all('td')) == 10]
    
    t_head=[]
    for data in rows[0].find_all('td'):
        t_head.append(data.text)
        
    t_body=[]   
    for row in rows[1:]:
        r={}
        for index,data in enumerate(row.find_all('td')):
            r[t_head[index]]=data.text.strip()
        t_body.append(r) 

    with open("shares.csv",'w') as f:
        f.write(','.join(t_head)+ '\n')
        for row in t_body:
            f.write(','.join(row.values())+ '\n')
        print("Datas written to file successfully.")    

share_price_crawler()

Website Successfully Crawled.
Datas written to file successfully.


In [111]:
share_data=pd.read_csv("shares.csv")
share_data.head()

,S.N.,Traded Companies,No. Of Transaction,Max Price,Min Price,Closing Price,Traded Shares,Amount,Previous Closing,Difference Rs.
0,1,10% Prabhu Bank Debenture 2084,1,1020.0,1020.0,1020.0,55.0,56100.0,1028.0,-8.0
1,2,10.35% Agricultural Bank Debenture 2083,2,1015.1,1015.0,1015.0,260.0,263905.0,1032.0,-17.0
2,3,11% NIC Asia Debenture 082/83,1,1056.7,1056.7,1056.7,200.0,211340.0,1036.0,20.7
3,4,12 % Goodwill Finance Limited Debenture 2083,5,1140.0,1125.0,1140.0,671.0,759375.0,1140.0,0.0
4,5,12% ICFC Finance Limited Debenture 2083,9,1140.0,1118.0,1130.0,884.0,995770.0,1120.0,10.0
